In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [6]:
data_ref = client.dataset("openaq", project='bigquery-public-data')
data = client.get_dataset(data_ref)
tables = list(client.list_tables(data_ref))
for t in tables:
    print(t.table_id)

global_air_quality


In [7]:
table_ref = data_ref.table("global_air_quality")
table = client.get_table(table_ref)

In [8]:
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [15]:
#help(table)

In [11]:
table.num_rows

23033

In [16]:
query = """
    SELECT city
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
    """

In [17]:
query_job = client.query(query)

In [18]:
us_cities = query_job.to_dataframe()

In [20]:
len(us_cities)

3709

In [21]:
print(us_cities[:5])

                      city
0  Boston-Cambridge-Quincy
1                    BROWN
2                    BROWN
3                  Houston
4                  Houston


In [22]:
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale             88
Houston                             80
Los Angeles-Long Beach-Santa Ana    68
San Francisco-Oakland-Fremont       58
Riverside-San Bernardino-Ontario    58
Name: city, dtype: int64

In [23]:
query = """
    SELECT city, country
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
    """

In [24]:
query_job = client.query(query)
ans = query_job.to_dataframe()
ans.head()

,city,country
0,Boston-Cambridge-Quincy,US
1,BROWN,US
2,BROWN,US
3,Houston,US
4,Houston,US


In [25]:
query = """
    SELECT *
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
    """

In [26]:
query_job = client.query(query)
ans = query_job.to_dataframe()
ans.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Portsmouth,Boston-Cambridge-Quincy,US,no2,7.0000,2020-02-26 13:00:00+00:00,µg/m³,DEFRA,43.075370,-70.748020,1.0
1,Aberdeen,BROWN,US,no2,13.0000,2020-03-16 15:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,1.0
2,Aberdeen,BROWN,US,o3,39.0000,2020-03-16 17:00:00+00:00,µg/m³,DEFRA,45.468597,-98.494064,8.0
3,Freeport South Avenue I C1012,Houston,US,so2,0.0001,2016-03-06 18:00:00+00:00,ppm,Texas,28.964430,-95.354830,1.0
4,Lake Jackson C1016,Houston,US,no2,-0.0004,2016-03-06 18:00:00+00:00,ppm,Texas,29.043758,-95.472946,1.0


In [29]:
query = """
    SELECT score, title
    FROM `bigquery-public-data.hacker_news.full`
    WHERE type = "job"
    """

dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query_job = client.query(query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 395306740 bytes.


In [30]:
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

safe_query_job = client.query(query, job_config=safe_config)
safe_query_job.to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/kaggle-project-271505/queries/1205bb3d-2cdd-4ecc-813d-136f3c4b5285?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000. 395313152 or higher required.

(job ID: 1205bb3d-2cdd-4ecc-813d-136f3c4b5285)

           -----Query Job SQL Follows-----           

    |    .    |    .    |    .    |    .    |
   1:
   2:    SELECT score, title
   3:    FROM `bigquery-public-data.hacker_news.full`
   4:    WHERE type = "job"
   5:    
    |    .    |    .    |    .    |    .    |

In [31]:
ONE_GB = 1000*1000*1000

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

safe_query_job = client.query(query, job_config=safe_config)
job_post_scores = safe_query_job.to_dataframe()
job_post_scores.score.mean()

1.9072663226017859